In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import time
import pandas as pd

In [3]:
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# executable_path : /opt/ml/crawling/chromedriver
driver = webdriver.Chrome(executable_path='/opt/ml/input/final/chromedriver', chrome_options=chrome_options)
URL = "https://map.naver.com/v5/search/%EC%84%9C%EC%B4%88%EA%B5%AC%20%EC%9D%8C%EC%8B%9D%EC%A0%90?c=14137477.8183780,4506726.1284877,12,0,0,0,dh"  
driver.get(URL)

In [4]:
driver.switch_to.frame('searchIframe') #iframe 으로 이동하기 위한 코드
restaurants_list = pd.DataFrame()

In [5]:
from tqdm import tqdm
epoch = 6 # 6이 최대인듯
for _ in tqdm(range(epoch)):
    click_list = driver.find_elements(By.CLASS_NAME, "tzwk0")
    click_list[0].click() #아무거나 클릭을 해야 end키가 먹혀서 임의로 아무 클래스 클릭한거임
    for _ in range(10):
        driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END) # end키를 눌러서 아래까지 로딩
    time.sleep(2)
    click_list = driver.find_elements(By.CLASS_NAME, "tzwk0") # 식당 리스트
    name_list = driver.find_elements(By.CLASS_NAME, "TYaxT") # 식당 이름 리스트
    name_list = [user.text for user in name_list]
    tag_list = driver.find_elements(By.CLASS_NAME, "KCMnt") # 식당 종류 리스트
    tag_list = [user.text for user in tag_list]
    url_list = []
    for restaurant_link in click_list:
        restaurant_link.click()
        url_list.append(driver.current_url)
        time.sleep(0.5)
    restaurants_list2 = pd.DataFrame({'restaurant' : name_list, 'tag' : tag_list, 'url' : url_list}, dtype = str)
    restaurants_list = pd.concat([restaurants_list, restaurants_list2], axis = 0, sort=False)
    driver.find_elements(By.CLASS_NAME, "yUtES")[1].click() #다음페이지 클릭
    time.sleep(2)

100%|██████████| 6/6 [03:54<00:00, 39.13s/it]


In [7]:
restaurants_list[:10]

,restaurant,tag,url
0,원조부안집 교대역점,돼지고기구이,https://map.naver.com/v5/search/%EC%84%9C%EC%B...
1,카페 어느새봄,"카페,디저트",https://map.naver.com/v5/search/%EC%84%9C%EC%B...
2,텍사스데브라질 센트럴시티점,"스테이크,립",https://map.naver.com/v5/search/%EC%84%9C%EC%B...
3,샤브리샤브샤브,샤브샤브,https://map.naver.com/v5/search/%EC%84%9C%EC%B...
4,원조부안집 양재점,돼지고기구이,https://map.naver.com/v5/search/%EC%84%9C%EC%B...
5,우대옥,소고기구이,https://map.naver.com/v5/search/%EC%84%9C%EC%B...
6,하이디라오 서초지점,중식당,https://map.naver.com/v5/search/%EC%84%9C%EC%B...
7,슈퍼두퍼 강남점,햄버거,https://map.naver.com/v5/search/%EC%84%9C%EC%B...
8,바다당,생선회,https://map.naver.com/v5/search/%EC%84%9C%EC%B...
9,설마중,"육류,고기요리",https://map.naver.com/v5/search/%EC%84%9C%EC%B...


In [11]:
restaurants_list.to_csv('./save_csv/restaurant.csv', index=False)